In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import model_from_json
from keras.models import Sequential
from keras.preprocessing.image import img_to_array, load_img
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.utils import np_utils
%matplotlib inline

#csvのファイルパスを記載
csv_filepath = 'filename.csv'

#csvからファイルパスを読み込む
csv_file = pd.read_csv(csv_filepath, encoding="UTF-8")#もしくわUTF-8

#csvに記載しているファイルパス、ラベル、総数を格納
fnames = csv_file['filepath']
labels = csv_file['label']
fnames_total = len(fnames)

#データセットを格納する変数の初期化
img_train = np.zeros((fnames_total,32,32,3))

#データセットの格納
#前処理する場合はここでやる
for i,fname in enumerate(fnames):
    temp_img = load_img(fname, target_size=(32,32))
#    plt.imshow(temp_img)
#    plt.show()

    #画像を配列に変換し0-1で正規化
    temp_img_array = img_to_array(temp_img)
    temp_img_array = temp_img_array.astype('float32')/255.0
    img_train[i] = temp_img_array

#正解ラベルをOne-Hot表現に変換
label_train = np_utils.to_categorical(labels,10)

#表示
#print(fnames)
#print(label_train)
#print(fnames_total)
#print("-----------------------")
#print(hoge)

In [ ]:
#モデルを構築
model=Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

history=model.fit(img_train,label_train,batch_size=128,nb_epoch=20,verbose=1,validation_split=0.1)

#モデルと重みを保存
json_string=model.to_json()
open('Anime_cnn.json',"w").write(json_string)
model.save_weights('Anime_cnn.h5')

#モデルの表示
model.summary()


C:\Users\calut\Anaconda3\envs\aidemy\lib\site-packages\ipykernel_launcher.py:26: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 851 samples, validate on 95 samples
Epoch 1/20
851/851 [==============================] - 7s 8ms/step - loss: 2.2799 - acc: 0.1340 - val_loss: 2.7992 - val_acc: 0.0000e+00
Epoch 2/20
851/851 [==============================] - 7s 8ms/step - loss: 2.1622 - acc: 0.1763 - val_loss: 4.1951 - val_acc: 0.0000e+00
Epoch 3/20
851/851 [==============================] - 6s 8ms/step - loss: 1.9043 - acc: 0.2973 - val_loss: 6.2163 - val_acc: 0.0000e+00
Epoch 4/20
851/851 [==============================] - 7s 8ms/step - loss: 1.4734 - acc: 0.4536 - val_loss: 7.6248 - val_acc: 0.0000e+00
Epoch 5/20
851/851 [==============================] - 7s 9ms/step - loss: 1.1881 - acc: 0.5546 - val_loss: 6.7975 - val_acc: 0.0000e+00
Epoch 6/20
851/851 [==============================] - 6s 8ms/step - loss: 0.9892 - acc: 0.6710 - val_loss: 7.0837 - val_acc: 0.0000e+00
Epoch 7/20
851/851 [==============================] - 6s 7ms/step - loss: 0.8514 - acc: 0.6992 - val_loss: 7.1164 - val_acc: 0.0000e+00
Epo